In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython import display
import pymongo
import random
import numpy as np
from bokeh.io import output_notebook
from climatools.cliradlw.cliradlw_latest import *
from climatools.html.html import *
output_notebook()
pd.set_option('max_colwidth', -1)

Loading BokehJS ...

In [2]:
CLIENT = pymongo.MongoClient(host='localhost', port=27017)

In [3]:
LINKS = pd.DataFrame()

In [4]:
def markdown_anchor(dlbl=None, dcli=None, link=None):  
    band, molecule, atmpro = [dcli.param.get(p) for p in ('band', 'molecule', 'atmpro')]
    s_band = f'''{band}'''                                                      
    s_conc = f'''{molecule}'''                                                                       
    s_atmpro = f'''{atmpro}'''                                                                       
    s = f'{s_band} {s_conc} {s_atmpro}'                                             
    s_anchor = ''.join(list(np.random.choice(range(9), 11).astype(str)))
    
    if isinstance(link, pd.DataFrame):
        id_anchor = f'''<a id="{s_anchor}"></a>'''
        href_anchor = f'''<a href="#{s_anchor}">{atmpro}</a>'''
        print(s_anchor)
        print(id_anchor)
        print(href_anchor)
        display.display(display.Markdown(id_anchor))
        display.display(display.Markdown(s))
        try:
            print(':O')
            print(molecule, band, href_anchor)
            link.loc[f"{molecule}", f"{band}"] = link.loc[f"{molecule}", f"{band}"] + (' ' + href_anchor)
        except KeyError:
            link.loc[f"{molecule}", f"{band}"] = href_anchor
    else:
        display.display(display.Markdown(s))

def show_results(dlbl=None, dcli=None, link=None):                                                                            
    display.display(display.Markdown('-----------------------------'))                                             
    markdown_anchor(dlbl=dlbl, dcli=dcli, link=link)                                                                                                                                                          
    # Parameters                                                                                                   
    show_parameters(dlbl=dlbl, dcli=dcli)                                                                                                                                                                          
    # Plot cooling rate profiles                                                                                   
    show_cools(dlbl=dlbl, dcli=dcli)                                                                                                                                                                                      
    display.display(display.Markdown('--------------------------'))

In [28]:
display.display(display.HTML(LINKS.fillna('').to_html(escape=False)))

,[3],[4]
{'co2': 0.0008},mls,mls


# 2 x $CO_2$

In [6]:
MOL = 'co2'
CONC = 2 * 400e-6
COMMITNUMBER = '1013f91'
NV, DV = 1000, .001
ATMPRO = dict(mls=294, saw=257, trp=300)

## Atmosphere compositions

We first set the concentration of CO2 to that double of the usual 4e-6 ppmv.  Then we split the overall composition into those where CO2 overlaps with other gases and those where only CO2 is present.

In [7]:
atmcomp = AtmComposition.cliradlw_nongreys()
atmcomp.gasconcs['co2'] = CONC
atmcomp.gasinbands = {b:gs for b, gs in atmcomp.gasinbands.items() if MOL in gs}

In [8]:
atmcomps_olp = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b: gs}) 
                for b, gs in atmcomp.gasinbands.items() if len(gs) > 1]

In [9]:
atmcomps_bft = [AtmComposition(gasconcs=atmcomp.gasconcs, gasinbands={b:[MOL]}) for b, _ in atmcomp.gasinbands.items()]

In [10]:
## CO2 only case

In [20]:
atmpro = 'mls'

In [21]:
atmbft = atmcomps_bft[1]

In [22]:
parm_cli = atmbft.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])

In [23]:
data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)

In [24]:
parms_lbl = atmbft.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], nv=NV, dv=DV)

In [25]:
parms_lbl[0].avail_commits(collection=CLIENT.lblnew.bestfit_lw)

['a22ab94']

In [26]:
data_lbl = parms_lbl[0].modeldata_pymongo(collection=CLIENT.lblnew.bestfit_lw)

In [27]:
show_results(dlbl=data_lbl, dcli=data_cli, link=LINKS)

-----------------------------

01880782230
<a id="01880782230"></a>
<a href="#01880782230">mls</a>


<a id="01880782230"></a>

[4] {'co2': 0.0008} mls

:O
{'co2': 0.0008} [4] <a href="#01880782230">mls</a>


Run parameters  \
                                CLIRAD   
atmpro                 mls               
band                   [4]               
commitnumber           1013f91           
conc                   -                 
dv                     -                 
klin                   -                 
molecule               {'co2': 0.0008}   
ng_adju                -                 
ng_refs                -                 
nv                     -                 
option_compute_btable  -                 
option_compute_ktable  -                 
option_wgt_flux        -                 
option_wgt_k           -                 
ref_pts                -                 
tsfc                   294               
vmax                   -                 
vmin                   -                 
w_diffuse              -                 
wgt                    -                 

                                                                                   \
                                                                           LBLNEW   
atmpro                 mls                                                          
band                   3b                                                           
commitnumber           a22ab94                                                      
conc                   0.0008                                                       
dv                     0.001                                                        
klin                   0                                                            
molecule               co2                                                          
ng_adju                [0, 0]                                                       
ng_refs                [5, 4]                                                       
nv                     1000                                                         
option_compute_btable  0                                                            
option_compute_ktable  0                                                            
option_wgt_flux        1                                                            
option_wgt_k           1                                                            
ref_pts                [[1, 250], [10, 250]]                                        
tsfc                   294                                                          
vmax                   720                                                          
vmin                   620                                                          
w_diffuse              [[1.66, 1.66, 1.66, 1.66, 1.66], [1.66, 1.66, 1.66, 1.66]]   
wgt                    [[0, 0.5, 0.4, 0.6, 0.65], [0.7, 0.7, 0.55, 0.6]]            

                                                                   Fit parameters  
                                                                        (co2, 3b)  
atmpro                 mls                                                         
band                   3b                                                          
commitnumber           a22ab94                                                     
conc                   0.0004                                                      
dv                     0.001                                                       
klin                   0                                                           
molecule               co2                                                         
ng_adju                [0, 0]                                                      
ng_refs                [5, 4]                                                      
nv                     1000                                                        
option_compute_btable  1                                                           
option_compute_ktable  1                                                           
option_wgt_flux        1                            

--------------------------

In [19]:
show_results(dlbl=data_lbl, dcli=data_cli, link=LINKS)

-----------------------------

08138157864
<a id="08138157864"></a>
<a href="#08138157864">mls</a>


<a id="08138157864"></a>

[3] {'co2': 0.0008} mls

:O
{'co2': 0.0008} [3] <a href="#08138157864">mls</a>


Run parameters  \
                                CLIRAD   
atmpro                 mls               
band                   [3]               
commitnumber           1013f91           
conc                   -                 
dv                     -                 
klin                   -                 
molecule               {'co2': 0.0008}   
ng_adju                -                 
ng_refs                -                 
nv                     -                 
option_compute_btable  -                 
option_compute_ktable  -                 
option_wgt_flux        -                 
option_wgt_k           -                 
ref_pts                -                 
tsfc                   294               
vmax                   -                 
vmin                   -                 
w_diffuse              -                 
wgt                    -                 

                                                                                 \
                                                                         LBLNEW   
atmpro                 mls                                                        
band                   3a                                                         
commitnumber           a22ab94                                                    
conc                   0.0008                                                     
dv                     0.001                                                      
klin                   6.37556e-24                                                
molecule               co2                                                        
ng_adju                [0, 0, 0]                                                  
ng_refs                [3, 2, 4]                                                  
nv                     1000                                                       
option_compute_btable  0                                                          
option_compute_ktable  0                                                          
option_wgt_flux        1                                                          
option_wgt_k           1                                                          
ref_pts                [[1, 250], [10, 250], [500, 250]]                          
tsfc                   294                                                        
vmax                   620                                                        
vmin                   540                                                        
w_diffuse              [[1.6, 1.6, 1.7], [1.75, 1.75], [1.55, 1.55, 1.6, 1.85]]   
wgt                    [[0.7, 0.3, 0.7], [0.7, 0.6], [0.4, 0.5, 0.8, 0.95]]       

                                                                 Fit parameters  
                                                                      (co2, 3a)  
atmpro                 mls                                                       
band                   3a                                                        
commitnumber           a06b618                                                   
conc                   0.0004                                                    
dv                     0.001                                                     
klin                   6.37556e-24                                               
molecule               co2                                                       
ng_adju                [0, 0, 0]                                                 
ng_refs                [3, 2, 4]                                                 
nv                     1000                                                      
option_compute_btable  0                                                         
option_compute_ktable  1                                                         
option_wgt_flux        1                                                         
option_wgt_k           1                  

--------------------------

In [298]:
show_results(dlbl=data_lbl, dcli=data_cli, link=LINKS)

-----------------------------

34160112838
<a id="34160112838"></a>
<a href="#34160112838">saw</a>


<a id="34160112838"></a>

[3] {'co2': 0.0008} saw

:O
{'co2': 0.0008} [3] <a href="#34160112838">saw</a>


Run parameters  \
                                CLIRAD   
atmpro                             saw   
band                               [3]   
commitnumber                   1013f91   
conc                                 -   
dv                                   -   
klin                                 -   
molecule               {'co2': 0.0008}   
ng_adju                              -   
ng_refs                              -   
nv                                   -   
option_compute_btable                -   
option_compute_ktable                -   
option_wgt_flux                      -   
option_wgt_k                         -   
ref_pts                              -   
tsfc                               257   
vmax                                 -   
vmin                                 -   
w_diffuse                            -   
wgt                                  -   

                                                                          \
                                                                  LBLNEW   
atmpro                                                               saw   
band                                                                  3a   
commitnumber                                                     a22ab94   
conc                                                              0.0008   
dv                                                                 0.001   
klin                                                         6.37556e-24   
molecule                                                             co2   
ng_adju                                                        [0, 0, 0]   
ng_refs                                                        [3, 2, 4]   
nv                                                                  1000   
option_compute_btable                                                  0   
option_compute_ktable                                                  0   
option_wgt_flux                                                        1   
option_wgt_k                                                           1   
ref_pts                                [[1, 250], [10, 250], [500, 250]]   
tsfc                                                                 257   
vmax                                                                 620   
vmin                                                                 540   
w_diffuse              [[1.6, 1.6, 1.7], [1.75, 1.75], [1.55, 1.55, 1...   
wgt                    [[0.7, 0.3, 0.7], [0.7, 0.6], [0.4, 0.5, 0.8, ...   

                                                          Fit parameters  
                                                               (co2, 3a)  
atmpro                                                               mls  
band                                                                  3a  
commitnumber                                                     a06b618  
conc                                                              0.0004  
dv                                                                 0.001  
klin                                                         6.37556e-24  
molecule                                                             co2  
ng_adju                                                        [0, 0, 0]  
ng_refs                                                        [3, 2, 4]  
nv                                                                  1000  
option_compute_btable                                                  0  
option_compute_ktable                                                  1  
option_wgt_flux                                                        1  
option_wgt_k                                                           1  
ref_pts                                [(1, 250), (10, 250), (500, 250)]  
tsfc                                                                 294  
vmax                                                                 620  
v

--------------------------

In [177]:
# CO2 overlapped with other gases

In [178]:
atmpro = 'mls'

In [179]:
atmolp = atmcomps_olp[0]

In [180]:
parm_cli = atmolp.to_cliradparam(commitnumber=COMMITNUMBER, atmpro=atmpro, tsfc=ATMPRO[atmpro])

In [181]:
data_cli = parm_cli.modeldata_pymongo(collection=CLIENT.cliradnew.lw)

In [182]:
parms_lbl = atmolp.to_lblnewparam(bestfit_values=True, atmpro=atmpro, tsfc=ATMPRO[atmpro], dv=DV, nv=NV)

In [183]:
parms_lbl

[<class 'climatools.parameters.LBLnewOverlapParam'>
 atmpro: mls
 band: 3a
 commitnumber: None
 dv: 0.001
 molecule: {'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07}
 nv: 1000
 tsfc: 294]

In [184]:
parms_lbl[0].avail_commits(collection=CLIENT.lblnew.overlap_lw)

['a22ab94']

In [185]:
data_lbl = parms_lbl[0].modeldata_pymongo(collection=CLIENT.lblnew.overlap_lw)

In [186]:
show_results(dlbl=data_lbl, dcli=data_cli, link=LINKS)

-----------------------------

50614075515
<a id="50614075515"></a>
<a href="#50614075515">mls</a>


<a id="50614075515"></a>

[3] {'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07} mls

Run parameters  \
                                                                 CLIRAD   
atmpro                                                              mls   
band                                                                [3]   
commitnumber                                                    1013f91   
conc                                                                  -   
dv                                                                    -   
klin                                                                  -   
molecule               {'h2o': 'atmpro', 'co2': 0.0008, 'n2o': 3.2e-07}   
ng_adju                                                               -   
ng_refs                                                               -   
nv                                                                    -   
option_compute_btable                                                 -   
option_compute_ktable                                                 -   
option_wgt_flux                                                       -   
option_wgt_k                                                          -   
ref_pts                                                               -   
tsfc                                                                294   
vmax                                                                  -   
vmin                                                                  -   
w_diffuse                                                             -   
wgt                                                                   -   

                                                                         \
                                                                 LBLNEW   
atmpro                                                              mls   
band                                                                 3a   
commitnumber                                                    a22ab94   
conc                                                                  -   
dv                                                                0.001   
klin                                                                  -   
molecule               {'co2': 0.0008, 'h2o': 'atmpro', 'n2o': 3.2e-07}   
ng_adju                                                               -   
ng_refs                                                               -   
nv                                                                 1000   
option_compute_btable                                                 -   
option_compute_ktable                                                 -   
option_wgt_flux                                                       -   
option_wgt_k                                                          -   
ref_pts                                                               -   
tsfc                                                                294   
vmax                                                                  -   
vmin                                                                  -   
w_diffuse                                                             -   
wgt                                                                   -   

                                                          Fit parameters  \
                                                               (co2, 3a)   
atmpro                                                               mls   
band                                                                  3a   
commitnumber                                                     a06b618   
conc                                                              0.0004   
dv                                                                 0.001   
klin                                                         6.37556e-24   
molecule                                                             co2   
ng_adju                                                        [0, 0, 0

--------------------------

## Fin

In [187]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
